# Test 11 (paper) - Experimento 8 (PT)

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 11"

### Hiperparametros
* **epoch = 120**
    * steps x epoch = 146 (lotes de imagenes)
    * batch = 2
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* **Mini-mask shape: 28x28**
* **RPN anchor scales: (8, 16, 32, 64, 128)**
* Tasa de aprendizaje: 0.001
* **imagenes = 305**
    * entrenamiento 70% = 214
    * validacion 30% = 91
* etiquetas = 9140
* **resolucion = 1920 x 1080**
* etiquetas = bounding box formato VOC XML
* **numero de clases = 2 (arandano, arandano-maduro)**
* **data augmentation = false**


## Comprobar directorio principal

In [8]:
!pwd && ls -l

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport
total 501708
-rw-r--r--  1 root root      1095 Aug  9 19:45 LICENSE
-rw-r--r--  1 root root        58 Aug  9 19:45 MANIFEST.in
-rw-r--r--  1 root root     13771 Aug  9 19:45 README.md
drwxr-xr-x  3 root root      4096 Aug  7 02:27 app
drwxr-xr-x  3 root root     12288 Sep 16 11:44 arandano_cfg_test_11_fase_2_20220916T0336
drwxr-xr-x  3 root root     12288 Sep 17 07:27 arandano_cfg_test_11_fase_2_20220917T0007
drwxr-xr-x  4 root root     20480 Sep 11 22:26 arandano_cfg_test_5_20220907T0914
drwxr-xr-x  3 root root     12288 Sep  9 15:19 arandano_cfg_test_5_20220909T0812
drwxr-xr-x  5 root root     24576 Sep 18 21:32 arandano_cfg_test_5_2_20220917T0844
drwxr-xr-x  3 root root     28672 Sep 15 00:32 arandano_cfg_test_5_fase_2_20220913T0749
drwxr-xr-x  3 root root     12288 Sep 20 07:32 arandano_cfg_test_5_fase_2_2_20220919T0845
drwxr-xr-x  3 root root     12288 Sep 13 02:17 arandano_cfg_test_6_20220912T1950
drwxr-xr-x  3 root root     12288 Sep

# Importar bibliotecas

In [9]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

import imgaug.augmenters as iaa

# Fase 2 - Entrenamiento con dos clases y etiquetas de Bounding Box

En este entremamiento se utiliza un conjunto de datos simple con imágenes etiquetadas con cuadros delimitadores y una clase llamada 'Daño'. En la siguiente sección se encuentra el código para el entrenamiento del modelo. Se incluyen comentarios para describir mejor el flujo del programa.

In [10]:
class DatasetArandanos(Dataset):
    
    # la funcion load_dataset es usada para cargar el dataset de entrenamiento y test
    def load_dataset(self, dataset_dir, is_train=True):
        
        # se agrega una clase que se necesita para clasificar, en este caso arandano
        # self.add_class('source', 'class id', 'class name')
        self.add_class("dataset", 1, "arandano")
        self.add_class("dataset", 2, "arandano-maduro")
        #self.add_class("dataset", 3, "arandano-pinton")
        #self.add_class("dataset", 4, "arandano-semi")
        
        # se concatena dataset_dir con /images y /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train sera Verdadero si se esta entrenando el modelo y Falso si se esta testeando el modelo
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # se usa para omitir los últimos 4 caracteres: '.jpg' (en class_id.jpg)
            
            # si is_train es Verdadero se omiten todas las imágenes con id mayor que e iguales a 11074
            # aproximadamente el 80% del conjunto de datos es para entrenamiento
            if is_train and int(image_id) >= 40450 :
                #print("image_id: ", image_id)
                continue
             
            # si is_train no es Verdadero se omiten todas las imágenes con id menores a 11074
            if not is_train and int(image_id) < 40450:
                continue
            
            # se declara la ruta de la imagen y la ruta de las etiquetas 
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # usando la función add_image se pasan image_id, image_path y ann_path para que la 
            # imagen actual se agregue al conjunto de datos para entrenamiento o prueba
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # funcino usada para extraer bouding boxes desde archivos etiquetados 
    def extract_boxes(self, filename):

        # se puede ver en las imágenes que estan etiquetadas, como se extraen los valores de ancho, alto y bndbox

        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>

        # </annotation>
        
        # para analizar los archivos .xml
        tree = ElementTree.parse(filename)
        
        # para obtener la raíz del archivo xml
        root = tree.getroot()
        
        # se agregan todas las coordenadas x, y en boxes para todas las instancias de un objeto
        boxes = list()
        
        # se encuentran todos los atributos con el nombre bndbox que existan para cada ground truth en la imagen
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
                        
            #quita todas las imagenes no etiquetadas
            if name=='arandano' or name=='arandano-maduro':
                boxes.append(coors)

        # extraer ancho y alto de la imagen
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # retorna boxes-> list, width-> int y height-> int 
        return boxes, width, height
    
    # esta función llama al método extract_boxes y se usa para cargar una máscara para cada instancia en una imagen 
    # devuelve una máscara booleana con las siguientes dimensiones ancho * alto * instancias
    def load_mask(self, image_id):
        
        # info apunta al image_id actual 
        info = self.image_info[image_id]
        
        # se obtiene la ruta de anotación de image_id que es dataset_dir/annots/image_id.xml 
        path = info['annotation']
        
         # se llama al método extract_boxes (arriba) para obtener bndbox del archivo .xml
        boxes, w, h = self.extract_boxes(path)
        
        # se crea una cantidad de len(boxes) de mascaras de alto 'h' y ancho 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = list()
        
        ## se recorren todos los boxes y generamos máscaras (máscara de bndbox) y class id para cada instancia
        # las máscaras tendrán forma rectangular ya que hemos usado bndboxes para etiquetas
        # por ejemplo: si 2.jpg tiene tres objetos, tendremos las siguientes máscaras y class_ids.
        
        # 000000000 000000000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000000000 022200000 000000000 000000000
        #    1         2          3         1<- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            # box[4] will have the name of the class for a particular damage
            if (box[4] == 'arandano'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('arandano'))
            elif(box[4] == 'arandano-maduro'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('arandano-maduro')) 
                
        # retorna mascaras y class_ids como arreglo
        return masks, asarray(class_ids, dtype='int32')
    
    # esta funciones toma el image_id y retorna la ruta de la imagen 
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']

In [11]:

# damage configuration class, you can change values of hyper parameters here
class ConfigArandanos(Config):

    # nombre de la configuracion
    NAME = "arandano_cfg_test_11_fase_2_2_"    
    
    # clase arandano + clase background + 4 clases
    NUM_CLASSES = 1 + 2
    
    # pasos por epoch y confianza minima    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    STEPS_PER_EPOCH = 146  # por epoch se entrenaran 61 lotes de 5 imagenes, dataset = 305

    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 

    #DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0    
    DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

    IMAGES_PER_GPU = 2
    
    MINI_MASK_SHAPE = (28, 28)
    
ConfigArandanos().display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DEVICE                         /gpu:0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE 

### Entrenamiento

In [12]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [13]:
#pesos = 'mask_rcnn_coco.h5'
#pesos = 'mask_rcnn_damage_cfg_0049.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
pesos = 'arandano_cfg_test_6_fase_2_2_20220920T0813/mask_rcnn_arandano_cfg_test_6_fase_2_2__0120.h5'

conjunto_datos = 'customImages/test_11_fase_2'


In [14]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()

# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)

# cargar pesos del modelo 
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# start the training of model
# you can change epochs and layers (head or all)
#model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=100, layers='all', augmentation=augmentation)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=120, layers='all')



Starting at epoch 0. LR=0.001

Checkpoint Path: ./arandano_cfg_test_11_fase_2_2_20220921T0813/mask_rcnn_arandano_cfg_test_11_fase_2_2__{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b     

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/120


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier_1/concat_grad/GatherV2_5:0", 

146/146 [==============================] - ETA: 0s - batch: 72.5000 - size: 2.0000 - loss: 2.9067 - rpn_class_loss: 0.2281 - rpn_bbox_loss: 1.2188 - mrcnn_class_loss: 0.1731 - mrcnn_bbox_loss: 0.7808 - mrcnn_mask_loss: 0.5059

...

146/146 [==============================] - 707s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 2.9067 - rpn_class_loss: 0.2281 - rpn_bbox_loss: 1.2188 - mrcnn_class_loss: 0.1731 - mrcnn_bbox_loss: 0.7808 - mrcnn_mask_loss: 0.5059 - val_loss: 2.3238 - val_rpn_class_loss: 0.3101 - val_rpn_bbox_loss: 0.7490 - val_mrcnn_class_loss: 0.1814 - val_mrcnn_bbox_loss: 0.5669 - val_mrcnn_mask_loss: 0.5164
Epoch 2/120
146/146 [==============================] - 525s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 1.8195 - rpn_class_loss: 0.1016 - rpn_bbox_loss: 0.6504 - mrcnn_class_loss: 0.1324 - mrcnn_bbox_loss: 0.4503 - mrcnn_mask_loss: 0.4847 - val_loss: 2.1404 - val_rpn_class_loss: 0.2203 - val_rpn_bbox_loss: 0.7326 - val_mrcnn_class_loss: 0.1710 - val_mrcnn_bbox_loss: 0.5240 - val_mrcnn_mask_loss: 0.4925
Epoch 3/120
146/146 [==============================] - 630s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 1.6549 - rpn_class_loss: 0.0780 - rpn_bbox_loss: 0.5959 - mrcnn_class_loss: 0.1531 -

Epoch 21/120
146/146 [==============================] - 582s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.8682 - rpn_class_loss: 0.0352 - rpn_bbox_loss: 0.1033 - mrcnn_class_loss: 0.2027 - mrcnn_bbox_loss: 0.1779 - mrcnn_mask_loss: 0.3493 - val_loss: 1.9378 - val_rpn_class_loss: 0.2214 - val_rpn_bbox_loss: 0.6124 - val_mrcnn_class_loss: 0.4109 - val_mrcnn_bbox_loss: 0.3312 - val_mrcnn_mask_loss: 0.3621
Epoch 22/120
146/146 [==============================] - 580s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.8225 - rpn_class_loss: 0.0340 - rpn_bbox_loss: 0.0980 - mrcnn_class_loss: 0.1818 - mrcnn_bbox_loss: 0.1666 - mrcnn_mask_loss: 0.3421 - val_loss: 2.0047 - val_rpn_class_loss: 0.2424 - val_rpn_bbox_loss: 0.6697 - val_mrcnn_class_loss: 0.3945 - val_mrcnn_bbox_loss: 0.3225 - val_mrcnn_mask_loss: 0.3755
Epoch 23/120
146/146 [==============================] - 531s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.7999 - rpn_class_loss: 0.0330 - rpn_bbox_loss: 0.0915 - mrcnn_class

Epoch 41/120
146/146 [==============================] - 577s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.5676 - rpn_class_loss: 0.0211 - rpn_bbox_loss: 0.0435 - mrcnn_class_loss: 0.1086 - mrcnn_bbox_loss: 0.0992 - mrcnn_mask_loss: 0.2951 - val_loss: 2.1278 - val_rpn_class_loss: 0.3046 - val_rpn_bbox_loss: 0.7827 - val_mrcnn_class_loss: 0.3965 - val_mrcnn_bbox_loss: 0.2931 - val_mrcnn_mask_loss: 0.3509
Epoch 42/120
146/146 [==============================] - 660s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.5512 - rpn_class_loss: 0.0203 - rpn_bbox_loss: 0.0412 - mrcnn_class_loss: 0.1040 - mrcnn_bbox_loss: 0.0969 - mrcnn_mask_loss: 0.2889 - val_loss: 2.1284 - val_rpn_class_loss: 0.2961 - val_rpn_bbox_loss: 0.7588 - val_mrcnn_class_loss: 0.4327 - val_mrcnn_bbox_loss: 0.2897 - val_mrcnn_mask_loss: 0.3511
Epoch 43/120
146/146 [==============================] - 520s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.5461 - rpn_class_loss: 0.0210 - rpn_bbox_loss: 0.0449 - mrcnn_class

Epoch 61/120
146/146 [==============================] - 657s 5s/step - batch: 72.5000 - size: 2.0000 - loss: 0.4357 - rpn_class_loss: 0.0139 - rpn_bbox_loss: 0.0328 - mrcnn_class_loss: 0.0649 - mrcnn_bbox_loss: 0.0679 - mrcnn_mask_loss: 0.2563 - val_loss: 2.1679 - val_rpn_class_loss: 0.3364 - val_rpn_bbox_loss: 0.8242 - val_mrcnn_class_loss: 0.3821 - val_mrcnn_bbox_loss: 0.2778 - val_mrcnn_mask_loss: 0.3475
Epoch 62/120
146/146 [==============================] - 574s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.4437 - rpn_class_loss: 0.0141 - rpn_bbox_loss: 0.0388 - mrcnn_class_loss: 0.0659 - mrcnn_bbox_loss: 0.0720 - mrcnn_mask_loss: 0.2529 - val_loss: 2.1375 - val_rpn_class_loss: 0.3106 - val_rpn_bbox_loss: 0.7752 - val_mrcnn_class_loss: 0.4304 - val_mrcnn_bbox_loss: 0.2750 - val_mrcnn_mask_loss: 0.3464
Epoch 63/120
146/146 [==============================] - 572s 4s/step - batch: 72.5000 - size: 2.0000 - loss: 0.4124 - rpn_class_loss: 0.0133 - rpn_bbox_loss: 0.0308 - mrcnn_class

KeyboardInterrupt: 